In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/CorpusforDDL_compute


In [2]:
import pandas as pd
import re
import unicodedata

In [3]:
%%capture
pip install -U ckip-transformers

In [4]:
import ckip_transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
ws_driver = CkipWordSegmenter(device=0)
pos_driver = CkipPosTagger()

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/407M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/407M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
# 讀入國語日報新聞 txt 檔
mandarin_daily_path = 'CorpusforDDL/國語日報新聞/MandarinDailyNews-20181217'
all_files = os.listdir(mandarin_daily_path)

mandarin_daily_corpus = []
for f in all_files:
  with open(f'{mandarin_daily_path}/{f}') as f:
      lines = f.readlines()
      mandarin_daily_corpus.append(lines)

In [6]:
# 前處理
def preprocess_mandarin_daily(string):

  clean_string = re.sub('[\n△◎]', '', string) # 移除換行符號和△◎符號
  clean_string = re.sub("【\w+】", "", clean_string) # 移除【記者xxx報導】
  splitted_strings = re.split(r'(?<=[。！？])', clean_string) # 將長句子用。！？斷開
  splitted_strings = [x for x in splitted_strings if len(x)>1] # 濾掉空字串和只有一個標點符號的字串

  return splitted_strings

In [7]:
test = """
【記者楊惠芳報導】
△小說家徐薏藍每年固定出兩本書之外，還在春秋兩季，
◎邀約文友，組成旅行團，周遊列國，觀賞世界各地美景。
今年開春後，她準備率團去帛琉，一窺海底祕密。目前報名的有五對夫婦及六位文友。
欲參加文友請早報名，以免向隅。
http://www.lttc.ntu.edu.tw。
"""

print(preprocess_mandarin_daily(test))

['小說家徐薏藍每年固定出兩本書之外，還在春秋兩季，邀約文友，組成旅行團，周遊列國，觀賞世界各地美景。', '今年開春後，她準備率團去帛琉，一窺海底祕密。', '目前報名的有五對夫婦及六位文友。', '欲參加文友請早報名，以免向隅。', 'http://www.lttc.ntu.edu.tw。']


In [8]:
mandarin_daily_corpus_preprocessed = []

for text in mandarin_daily_corpus:
  text_preprocessed = []
  for string in text:
    check_punctuation = re.search(r'[，。！？]', string) # 檢查字串是否包含，。！？
    check_chinese = re.search(r'[\u4e00-\u9fff]+', string) # 檢查字串是否包含中文字

    if check_punctuation and check_chinese:
      preprocessed = preprocess_mandarin_daily(string)
      text_preprocessed.append(preprocessed)

  mandarin_daily_corpus_preprocessed.append(text_preprocessed)

In [18]:
mandarin_daily_test = [item for sublist in mandarin_daily_corpus_preprocessed[0] for item in sublist]
mandarin_daily_test[:10]

['今天是中華民國九十年元旦，也是二十一世紀的第一天，陳水扁總統發表跨世紀談話，並且向全民祝賀新年。',
 '陳總統表示，過去政府的「戒急用忍」政策有當時的背景及其必要，但是未來政府將以「積極開放，有效管理」的新視野，規畫我們的新世紀經貿版圖。',
 '陳總統指出，二十一世紀，經濟競爭力依然是我們賴以生存的命脈。',
 '面對全球景氣循環的低潮以及包括美國、日本等國家經濟成長的向下修正，臺灣地區的經濟繁榮將再次面臨高難度的考驗，政府與民間要為最壞的可能做最好的準備。',
 '另外，因應我們即將加入世界貿易組織，陳總統說，政府有責任面對全新的經濟情勢，將包括兩岸經貿在內的各項課題，重新納入全球市場的考量。',
 '他說，在過去的「戒急用忍」之外，未來我們將以「積極開放，有效管理」的新視野，在知識經濟的既定方針之下，為新世紀經貿版圖做出宏觀的規畫，並且逐步加以落實。',
 '陳水扁總統說，面對過去半年多以來的政經情勢，國人應該都能深刻的體會，有穩定的政局才有經濟的繁榮，有穩定的政局也才有改革和進步。',
 '在兩岸問題方面，陳總統強調，依據中華民國憲法，「一個中國」原本並不是問題，也希望對岸能夠深入了解臺灣地區人民心中的疑慮在哪裡，尊重兩千三百多萬人民的意志。',
 '陳總統認為，兩岸原是一家人，也有共存共榮的相同目標，應該要相互體諒、相互提攜。',
 '他呼籲對岸的政府與領導人，尊重中華民國生存的空間與國際的尊嚴，公開放棄武力的威脅，以最大的氣度和前瞻的智慧，超越目前的僵局，從兩岸經貿與文化的統合開始著手，逐步建立兩岸之間的信任，進而共同尋求兩岸永久和平、政治統合的新架構。']

In [22]:
# 斷詞
import pickle
mandarin_daily_ws  = ws_driver(mandarin_daily_test)
with open('mandarin_daily_ws.pkl', 'wb') as f:
    pickle.dump(mandarin_daily_ws, f)

Inference: 100%|██████████| 161/161 [14:18<00:00,  5.33s/it]


In [ ]:
mandarin_daily_pos = pos_driver(mandarin_daily_ws)
